# Mean variance optimization with constraints

* here, long only constraints

* Using cvxopt 


In [76]:
from math import sqrt
from cvxopt import matrix
from cvxopt.blas import dot 
from cvxopt.solvers import qp, options
import pylab
from pandas import DataFrame

%matplotlib inline

# Data of the problem (same as efffront_exercise.ipynb)

In [67]:
r=0.02 # Cash rate

s1=0.07 # Vol
sr1=0.05 # Sharpe
s2=0.16 # Vol
sr2=0.25 # Sharpe
rho=-0.2 # Correlation
sigma=matrix([[s1**2,rho*s1*s2],[rho*s1*s2,s2**2]]) # Covariance matrix
#sigma=np.array([[s1**2,rho*s1*s2],[rho*s1*s2,s2**2]]) # Covariance matrix


mu1=sr1*s1 # Expected excess return
mu2=sr2*s2 # Expected excess return
mu=matrix([mu1,mu2]) # Vector of expected returns

lbd=5 # Risk aversion

print sigma
print mu

[ 4.90e-03 -2.24e-03]
[-2.24e-03  2.56e-02]

[ 3.50e-03]
[ 4.00e-02]



# Define the constraints of the problem

* All asset weights should be positive 

* The control variable only pertains of risky asset weights $(\pi_{1},\pi_{2})$

* Cash is positive if $\pi_{1}+\pi_{2} \leq 1$

* Check [this reference](http://cvxopt.org/userguide/coneprog.html#quadratic-programming) for the conventions of cvxopt

In [68]:
G = matrix(0.0, (3,2))
G[0,0] = -1.0
G[1,1] = -1.0
G[2,:]=[1.0,1.0]
h = matrix(0.0, (3,1))
h[2]=1

print G
print h

[-1.00e+00  0.00e+00]
[ 0.00e+00 -1.00e+00]
[ 1.00e+00  1.00e+00]

[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]



In [79]:
options['show_progress'] = False
sol = qp(lbd*sigma, -mu, G, h)

pistar= [1-sum(sol['x']), sol['x'][0],sol['x'][1]]

df = DataFrame(pistar, index=['Cash','Bonds','Equities'],columns=['Opt. weights'])
df

,Opt. weights
Cash,0.363836
Bonds,0.297622
Equities,0.338542
